In [1]:
import numpy as np
import pandas as pd


btc_usdt_path = '/Users/apple/Desktop/time-series/informer-validation/data/BTC_USDT_2023-2025_1m.csv'
fng_path = '/Users/apple/Desktop/time-series/informer-validation/data/fear_greed_index.csv'
fed_funds_path = '/Users/apple/Desktop/time-series/informer-validation/data/FEDFUNDS.csv'
vix_path = '/Users/apple/Desktop/time-series/informer-validation/data/VIXCLS.csv'

In [22]:
btc_usdt = pd.read_csv(btc_usdt_path)
btc_usdt['datetime'] = pd.to_datetime(btc_usdt['datetime'])
btc_usdt

,datetime,close,high,low,open,volume
0,2023-01-01 00:00:00+00:00,16543.67,16544.76,16538.45,16541.77,83.08143
1,2023-01-01 00:01:00+00:00,16539.31,16544.41,16538.48,16543.04,80.45300
2,2023-01-01 00:02:00+00:00,16536.43,16541.17,16534.52,16539.31,62.90197
3,2023-01-01 00:03:00+00:00,16533.65,16537.28,16531.00,16536.43,115.71894
4,2023-01-01 00:04:00+00:00,16535.38,16536.08,16527.51,16534.12,144.45369
...,...,...,...,...,...,...
1153285,2025-03-11 23:55:00+00:00,82836.14,82840.28,82746.16,82840.28,5.22269
1153286,2025-03-11 23:56:00+00:00,82821.16,82836.15,82790.73,82836.14,5.57161
1153287,2025-03-11 23:57:00+00:00,82822.21,82834.31,82803.41,82821.16,20.74109
1153288,2025-03-11 23:58:00+00:00,82878.30,82921.70,82822.21,82822.21,76.26531


In [23]:
fng = pd.read_csv(fng_path)
fng['datetime'] = pd.to_datetime(fng['datetime'])
fng

,datetime,value,value_classification
0,2018-02-01,30,Fear
1,2018-02-02,15,Extreme Fear
2,2018-02-03,40,Fear
3,2018-02-04,24,Extreme Fear
4,2018-02-05,11,Extreme Fear
...,...,...,...
2735,2025-08-02,55,Greed
2736,2025-08-03,53,Neutral
2737,2025-08-04,64,Greed
2738,2025-08-05,60,Greed


In [18]:
fed_funds = pd.read_csv(fed_funds_path)
fed_funds.rename(columns={'observation_date': 'datetime'}, inplace=True)
fed_funds['datetime'] = pd.to_datetime(fed_funds['datetime'])
fed_funds

,datetime,FEDFUNDS
0,2020-07-01,0.09
1,2020-08-01,0.10
2,2020-09-01,0.09
3,2020-10-01,0.09
4,2020-11-01,0.09
...,...,...
56,2025-03-01,4.33
57,2025-04-01,4.33
58,2025-05-01,4.33
59,2025-06-01,4.33


In [19]:
vix = pd.read_csv(vix_path)
vix.rename(columns={'observation_date': 'datetime'}, inplace=True)
vix['datetime'] = pd.to_datetime(vix['datetime'])
vix

,datetime,VIXCLS
0,2020-01-02,12.47
1,2020-01-03,14.02
2,2020-01-06,13.85
3,2020-01-07,13.79
4,2020-01-08,13.45
...,...,...
1453,2025-07-29,15.98
1454,2025-07-30,15.48
1455,2025-07-31,16.72
1456,2025-08-01,20.38


In [27]:
btc_usdt_columns = btc_usdt.columns.tolist()
fng_columns = fng.columns.tolist()
fed_funds_columns = fed_funds.columns.tolist()
vix_columns = vix.columns.tolist()


# Get interval frequency for each dataset
btc_usdt_frequency = '1 min'
fng_frequency = 'daily'
fed_funds_frequency = 'monthly'
vix_frequency = 'daily'

print(f"BTC_USDT columns: {btc_usdt_columns}, Frequency: {btc_usdt_frequency}")
print(f"Fear and Greed Index columns: {fng_columns}, Frequency: {fng_frequency}")
print(f"Fed Funds columns: {fed_funds_columns}, Frequency: {fed_funds_frequency}")
print(f"VIX columns: {vix_columns}, Frequency: {vix_frequency}")

BTC_USDT columns: ['datetime', 'close', 'high', 'low', 'open', 'volume'], Frequency: 1 min
Fear and Greed Index columns: ['datetime', 'value', 'value_classification'], Frequency: daily
Fed Funds columns: ['datetime', 'FEDFUNDS'], Frequency: monthly
VIX columns: ['datetime', 'VIXCLS'], Frequency: daily


In [35]:
import pandas as pd


# 1-min BTC/USDT
btc = pd.read_csv(btc_usdt_path, parse_dates=["datetime"])
btc = btc.sort_values("datetime")

# daily Fear & Greed
fng = (
    pd.read_csv(fng_path, parse_dates=["datetime"])
      .sort_values("datetime")
      .rename(columns={"value": "fng_value"})
)

# daily VIX
vix = (
    pd.read_csv(vix_path, parse_dates=["observation_date"])
      .rename(columns={
          "observation_date": "datetime",
          "VIXCLS": "vix_value",
      })
      .sort_values("datetime")
)

# Reindex to every calendar day, forward‐filling so Sat/Sun get Fri’s number
vix = (
    vix
    .set_index("datetime")
    .asfreq("D")                  # now has one row per calendar day
    .ffill()                      # fill missing days with last valid (Fri→Sat/Sun)
    .reset_index()
)


# monthly Fed Funds
fed = (
    pd.read_csv(fed_funds_path, parse_dates=["observation_date"])
      .sort_values("observation_date")
      .rename(columns={
          "observation_date": "datetime",
          "FEDFUNDS": "fed_funds_rate"
        })
)

# make all datetime columns naive
btc["datetime"] = btc["datetime"].dt.tz_localize(None)
fng["datetime"] = fng["datetime"].dt.tz_localize(None)
vix["datetime"] = vix["datetime"].dt.tz_localize(None)
fed["datetime"] = fed["datetime"].dt.tz_localize(None)


# ——— 2) Merge-as-of (backward fill) —————————————————————————————

# Merge fear & greed
merged = pd.merge_asof(
    btc,
    fng[["datetime", "fng_value"]],
    on="datetime",
    direction="backward",
    tolerance=pd.Timedelta("1D")
)

# Merge VIX
merged = pd.merge_asof(
    merged,
    vix[["datetime", "vix_value"]],
    on="datetime",
    direction="backward",
    tolerance=pd.Timedelta("1D")
)

# Merge Fed Funds
merged = pd.merge_asof(
    merged,
    fed[["datetime", "fed_funds_rate"]],
    on="datetime",
    direction="backward",
    tolerance=pd.Timedelta("31D")
)

# ——— 3) Forward-fill through each day/month ——————————————————————

# any minute after the last known value keeps that last value
merged[["fng_value", "vix_value", "fed_funds_rate"]] = (
    merged[["fng_value", "vix_value", "fed_funds_rate"]]
    .ffill()
)

# ——— 4) Inspect & save ——————————————————————————————————————


merged.to_csv("/Users/apple/Desktop/time-series/informer-validation/data/btc_with_macro.csv", index=False)
merged



,datetime,close,high,low,open,volume,fng_value,vix_value,fed_funds_rate
0,2023-01-01 00:00:00,16543.67,16544.76,16538.45,16541.77,83.08143,26.0,21.67,4.33
1,2023-01-01 00:01:00,16539.31,16544.41,16538.48,16543.04,80.45300,26.0,21.67,4.33
2,2023-01-01 00:02:00,16536.43,16541.17,16534.52,16539.31,62.90197,26.0,21.67,4.33
3,2023-01-01 00:03:00,16533.65,16537.28,16531.00,16536.43,115.71894,26.0,21.67,4.33
4,2023-01-01 00:04:00,16535.38,16536.08,16527.51,16534.12,144.45369,26.0,21.67,4.33
...,...,...,...,...,...,...,...,...,...
1153285,2025-03-11 23:55:00,82836.14,82840.28,82746.16,82840.28,5.22269,24.0,26.92,4.33
1153286,2025-03-11 23:56:00,82821.16,82836.15,82790.73,82836.14,5.57161,24.0,26.92,4.33
1153287,2025-03-11 23:57:00,82822.21,82834.31,82803.41,82821.16,20.74109,24.0,26.92,4.33
1153288,2025-03-11 23:58:00,82878.30,82921.70,82822.21,82822.21,76.26531,24.0,26.92,4.33
